<a href="https://colab.research.google.com/github/sjameelhassan/JameelHassan/blob/master/TextMining_codechallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.6MB/s 
     |████████████████████████████████| 552kB 38.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=d791e1a2a92ccbc8fe0fd13a9c850160359f6f909b031ce88beb5bee99e5175b
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=bb6341e8ef1747dc346bd3f8018a200b64b88d40fc813f9dad744b55ec048d09
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
# Import packages

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization

import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [42]:
# Importing Dataset

raw_data=pd.read_csv('/content/drive/My Drive/DataSets/articles1.csv', engine='python',error_bad_lines=False)

Skipping line 17535: field larger than field limit (131072)


In [43]:
#Download nltk stopwords and spacy model

import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.


**Prepare Stopwords**


In [0]:
# NLTK Stop words
## We have already downloaded the stopwords. Let’s import them and make it available in 'stopwords'

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [45]:
raw_data.head(5)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [46]:
len(raw_data['author'].unique()) # No of authors

3604

In [47]:
## Remove emails and newline characters and extra spaces - Data cleaning using regulat expressions

# Convert to list
data = raw_data.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['WASHINGTON — Congressional Republicans have a new fear when it comes to '
 'their health care lawsuit against the Obama administration: They might win. '
 'The incoming Trump administration could choose to no longer defend the '
 'executive branch against the suit, which challenges the administration’s '
 'authority to spend billions of dollars on health insurance subsidies for and '
 'Americans, handing House Republicans a big victory on issues. But a sudden '
 'loss of the disputed subsidies could conceivably cause the health care '
 'program to implode, leaving millions of people without access to health '
 'insurance before Republicans have prepared a replacement. That could lead to '
 'chaos in the insurance market and spur a political backlash just as '
 'Republicans gain full control of the government. To stave off that outcome, '
 'Republicans could find themselves in the awkward position of appropriating '
 'huge sums to temporarily prop up the Obama health care law, angerin

In [55]:
## Tokenize words and Clean-up text --> You need to break down each sentence into a list of words through tokenization, while clearing up all the messy text in the process
# Gensim’s simple_preprocess() is great for this.  

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['washington', 'congressional', 'republicans', 'have', 'new', 'fear', 'when', 'it', 'comes', 'to', 'their', 'health', 'care', 'lawsuit', 'against', 'the', 'obama', 'administration', 'they', 'might', 'win', 'the', 'incoming', 'trump', 'administration', 'could', 'choose', 'to', 'no', 'longer', 'defend', 'the', 'executive', 'branch', 'against', 'the', 'suit', 'which', 'challenges', 'the', 'administration', 'authority', 'to', 'spend', 'billions', 'of', 'dollars', 'on', 'health', 'insurance', 'subsidies', 'for', 'and', 'americans', 'handing', 'house', 'republicans', 'big', 'victory', 'on', 'issues', 'but', 'sudden', 'loss', 'of', 'the', 'disputed', 'subsidies', 'could', 'conceivably', 'cause', 'the', 'health', 'care', 'program', 'to', 'implode', 'leaving', 'millions', 'of', 'people', 'without', 'access', 'to', 'health', 'insurance', 'before', 'republicans', 'have', 'prepared', 'replacement', 'that', 'could', 'lead', 'to', 'chaos', 'in', 'the', 'insurance', 'market', 'and', 'spur', 'politic

In [56]:
## Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['washington', 'congressional', 'republicans', 'have', 'new', 'fear', 'when', 'it', 'comes', 'to', 'their', 'health_care', 'lawsuit', 'against', 'the', 'obama', 'administration', 'they', 'might', 'win', 'the', 'incoming', 'trump', 'administration', 'could', 'choose', 'to', 'no', 'longer', 'defend', 'the', 'executive_branch', 'against', 'the', 'suit', 'which', 'challenges', 'the', 'administration', 'authority', 'to', 'spend', 'billions', 'of', 'dollars', 'on', 'health_insurance', 'subsidies', 'for', 'and', 'americans', 'handing', 'house', 'republicans', 'big', 'victory', 'on', 'issues', 'but', 'sudden', 'loss', 'of', 'the', 'disputed', 'subsidies', 'could', 'conceivably', 'cause', 'the', 'health_care', 'program', 'to', 'implode', 'leaving', 'millions', 'of', 'people', 'without', 'access', 'to', 'health_insurance', 'before', 'republicans', 'have', 'prepared', 'replacement', 'that', 'could', 'lead', 'to', 'chaos', 'in', 'the', 'insurance', 'market', 'and', 'spur', 'political', 'backlash',

Remove Stopwords, Make Bigrams and Lemmatize

In [0]:
# # The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let’s call the functions in order

In [58]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['washington', 'congressional', 'republican', 'new', 'fear', 'come', 'health_care', 'lawsuit', 'obama', 'administration', 'may', 'win', 'incoming', 'trump', 'administration', 'could', 'choose', 'longer', 'defend', 'executive_branch', 'suit', 'challenge', 'administration', 'authority', 'spend', 'billion', 'dollar', 'health_insurance', 'subsidy', 'american', 'hand', 'house', 'republican', 'big', 'victory', 'issue', 'sudden', 'loss', 'dispute', 'subsidy', 'could', 'conceivably', 'health_care', 'program', 'implode', 'leave', 'million', 'people', 'access', 'health_insurance', 'republican', 'prepared', 'replacement', 'could', 'lead', 'chaos', 'insurance', 'market', 'spur', 'political', 'backlash', 'republican', 'gain', 'full', 'control', 'government', 'stave', 'outcome', 'republican', 'could', 'find', 'awkward', 'position', 'appropriate', 'huge_sum', 'temporarily', 'prop', 'obama', 'health_care', 'law', 'anger', 'conservative', 'voter', 'demand', 'end', 'law', 'year', 'twist', 'donald', 'tr

Create the Dictionary and Corpus needed for Topic Modeling

In [59]:
# # The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 13), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 4), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 4), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 3), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 5), (45, 3), (46, 1), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 8), (56, 2), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 5), (91, 1), (92, 1), (93, 1), (94, 1), (95, 3), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 7), (110, 4

In [60]:
id2word[1] # -> Gives what is word 1

'acknowledge'

In [66]:
# Human readable format of corpus (term-frequency)
# Genism creates bag of words in alphabetical order'

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('access', 1),
  ('acknowledge', 1),
  ('act', 1),
  ('administration', 13),
  ('advocate', 1),
  ('affordable_care', 1),
  ('ally', 1),
  ('american', 1),
  ('anger', 1),
  ('annual', 1),
  ('anticipate', 2),
  ('appeal', 4),
  ('appropriate', 2),
  ('appropriation', 1),
  ('approval', 1),
  ('aspect', 1),
  ('assert', 1),
  ('authority', 2),
  ('avoid', 1),
  ('awkward', 1),
  ('backlash', 1),
  ('behalf', 1),
  ('big', 1),
  ('billion', 1),
  ('blando', 1),
  ('broad', 1),
  ('capitol_hill', 1),
  ('cascade', 1),
  ('case', 4),
  ('central', 1),
  ('challenge', 1),
  ('champion', 1),
  ('chaos', 1),
  ('choose', 2),
  ('collyer', 3),
  ('columbia_circuit', 1),
  ('come', 3),
  ('comment', 1),
  ('committee', 1),
  ('complicate', 1),
  ('conceivably', 1),
  ('concept', 1),
  ('confidence', 1),
  ('confident', 1),
  ('congress', 5),
  ('congressional', 3),
  ('consequence', 1),
  ('conservative', 1),
  ('consider', 1),
  ('constitution', 3),
  ('consumer', 1),
  ('contend', 1),
  ('

**Model Building using LDA**

The chunksize I have assigned as 100 as its looks better tradeoff size both for computation time as well as for better estimation.

In [0]:
# Build LDA model
# We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
# Print the Keyword in the 10 keywords
# LDA model is built with 30 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.026*"show" + 0.018*"film" + 0.013*"star" + 0.012*"movie" + 0.010*"tv" + '
  '0.008*"video" + 0.007*"good" + 0.007*"play" + 0.007*"character" + '
  '0.006*"year"'),
 (1,
  '0.023*"new" + 0.017*"time" + 0.016*"may" + 0.014*"year" + 0.009*"first" + '
  '0.009*"write" + 0.007*"even" + 0.007*"become" + 0.006*"seem" + '
  '0.005*"find"'),
 (2,
  '0.017*"report" + 0.014*"email" + 0.014*"news" + 0.013*"say" + '
  '0.012*"intelligence" + 0.011*"source" + 0.011*"fbi" + 0.010*"investigation" '
  '+ 0.010*"official" + 0.009*"information"'),
 (3,
  '0.019*"law" + 0.015*"state" + 0.014*"would" + 0.013*"federal" + '
  '0.012*"case" + 0.011*"government" + 0.010*"court" + 0.010*"rule" + '
  '0.009*"administration" + 0.008*"legal"'),
 (4,
  '0.016*"country" + 0.012*"political" + 0.010*"leave" + 0.010*"party" + '
  '0.010*"world" + 0.010*"europe" + 0.008*"government" + 0.008*"power" + '
  '0.007*"leader" + 0.007*"remain"'),
 (5,
  '0.012*"city" + 0.008*"water" + 0.007*"day" + 0.007*"area" + 0.0

Compute Model Perplexity and Coherence Score

In [0]:
# in my experience, topic coherence score, in particular, has been more helpful.

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.548586100034228

Coherence Score:  0.5402410376935015


Model perplexity and topic coherence provides a convenient measure to judge how good a given topic model is. Topic coherence score, in particular, has been more helpful.

In [0]:
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

**Visualize the topics-keywords**

In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis
# pyLDAvis.gensim.prepare

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.045490 -0.062447       1        1  17.352654
8     -0.158922 -0.136606       2        1   7.923396
1      0.069874  0.011663       3        1   6.760636
3     -0.175782  0.087876       4        1   6.208057
13    -0.042022  0.137295       5        1   6.020715
2     -0.127234  0.059902       6        1   6.017263
12    -0.077359 -0.027702       7        1   5.842644
19    -0.006423 -0.051499       8        1   5.403323
5      0.099998 -0.096266       9        1   5.037143
16     0.118283 -0.101370      10        1   5.003458
9     -0.111897  0.005146      11        1   4.671262
17    -0.119117 -0.105994      12        1   4.127506
4     -0.100294 -0.060315      13        1   3.332158
0      0.184420 -0.138438      14        1   3.180185
11    -0.177698  0.039957      15        1   3.068642
7      0.026507  0.163744      16        1   2.659199
18     0.058776  0.200261      17        1   2.603852
14     0.146186  0.224035      18        1   2.265446
10     0.137630 -0.203415      19        1   1.882268
6      0.300565  0.054172      20        1   0.640202, topic_info=     Category           Freq        Term          Total  loglift  logprob
257   Default  179186.000000       trump  179186.000000  30.0000  30.0000
224   Default  213341.000000         say  213341.000000  29.0000  29.0000
214   Default   39998.000000  republican   39998.000000  28.0000  28.0000
841   Default   31832.000000      police   31832.000000  27.0000  27.0000
593   Default   63301.000000          go   63301.000000  26.0000  26.0000
...       ...            ...         ...            ...      ...      ...
511   Topic20    1040.596680       drive    7505.906738   3.0752  -4.5780
5482  Topic20     765.341064     mission    3044.432129   3.6704  -4.8852
2257  Topic20     708.112854  technology    5564.178223   2.9896  -4.9629
3688  Topic20     554.000000       crash    1884.955566   3.8266  -5.2084
2258  Topic20     487.720581        test    5869.643555   2.5633  -5.3358

[1326 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
23141     12  0.998507         aca
6933      13  0.104085    academic
6933      16  0.894945    academic
277        2  0.016006      accord
277        4  0.030054      accord
...      ...       ...         ...
1155      16  0.021903       young
24411     13  0.998401        yuan
93989      8  0.998256      zapata
74016     13  0.997795        zeta
27677     18  0.997328  zuckerberg

[3728 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 9, 2, 4, 14, 3, 13, 20, 6, 17, 10, 18, 5, 1, 12, 8, 19, 15, 11, 7])

**Interpretation**:

1. Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
2.A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
3. A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
4.if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.
5. Here I have successfully built a good looking topic model.

**NER**


In [0]:
## Import the packages

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

spacy_nlp = spacy.load("en")

nlp = spacy.load('en_core_web_sm')


In [50]:
data[:1]

['WASHINGTON — Congressional Republicans have a new fear when it comes to their health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for and Americans, handing House Republicans a big victory on issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been demanding an

In [20]:
## Second Test

artical='WASHINGTON — Congressional Republicans have a new fear when it comes to their health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for and Americans, handing House Republicans a big victory on issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been demanding an end to the law for years. In another twist, Donald J. Trump’s administration, worried about preserving executive branch prerogatives, could choose to fight its Republican allies in the House on some central questions in the dispute. Eager to avoid an ugly political pileup, Republicans on Capitol Hill and the Trump transition team are gaming out how to handle the lawsuit, which, after the election, has been put in limbo until at least late February by the United States Court of Appeals for the District of Columbia Circuit. They are not yet ready to divulge their strategy. “Given that this pending litigation involves the Obama administration and Congress, it would be inappropriate to comment,” said Phillip J. Blando, a spokesman for the Trump transition effort. “Upon taking office, the Trump administration will evaluate this case and all related aspects of the Affordable Care Act. ” In a potentially decision in 2015, Judge Rosemary M. Collyer ruled that House Republicans had the standing to sue the executive branch over a spending dispute and that the Obama administration had been distributing the health insurance subsidies, in violation of the Constitution, without approval from Congress. The Justice Department, confident that Judge Collyer’s decision would be reversed, quickly appealed, and the subsidies have remained in place during the appeal. In successfully seeking a temporary halt in the proceedings after Mr. Trump won, House Republicans last month told the court that they “and the ’s transition team currently are discussing potential options for resolution of this matter, to take effect after the ’s inauguration on Jan. 20, 2017. ” The suspension of the case, House lawyers said, will “provide the and his future administration time to consider whether to continue prosecuting or to otherwise resolve this appeal. ” Republican leadership officials in the House acknowledge the possibility of “cascading effects” if the payments, which have totaled an estimated $13 billion, are suddenly stopped. Insurers that receive the subsidies in exchange for paying costs such as deductibles and for eligible consumers could race to drop coverage since they would be losing money. Over all, the loss of the subsidies could destabilize the entire program and cause a lack of confidence that leads other insurers to seek a quick exit as well. Anticipating that the Trump administration might not be inclined to mount a vigorous fight against the House Republicans given the ’s dim view of the health care law, a team of lawyers this month sought to intervene in the case on behalf of two participants in the health care program. In their request, the lawyers predicted that a deal between House Republicans and the new administration to dismiss or settle the case “will produce devastating consequences for the individuals who receive these reductions, as well as for the nation’s health insurance and health care systems generally. ” No matter what happens, House Republicans say, they want to prevail on two overarching concepts: the congressional power of the purse, and the right of Congress to sue the executive branch if it violates the Constitution regarding that spending power. House Republicans contend that Congress never appropriated the money for the subsidies, as required by the Constitution. In the suit, which was initially championed by John A. Boehner, the House speaker at the time, and later in House committee reports, Republicans asserted that the administration, desperate for the funding, had required the Treasury Department to provide it despite widespread internal skepticism that the spending was proper. The White House said that the spending was a permanent part of the law passed in 2010, and that no annual appropriation was required — even though the administration initially sought one. Just as important to House Republicans, Judge Collyer found that Congress had the standing to sue the White House on this issue — a ruling that many legal experts said was flawed — and they want that precedent to be set to restore congressional leverage over the executive branch. But on spending power and standing, the Trump administration may come under pressure from advocates of presidential authority to fight the House no matter their shared views on health care, since those precedents could have broad repercussions. It is a complicated set of dynamics illustrating how a quick legal victory for the House in the Trump era might come with costs that Republicans never anticipated when they took on the Obama White House.'

document = spacy_nlp(artical)

for element in document.ents:
    print('Type: %s, Value: %s' % (element.label_, element))


Type: GPE, Value: WASHINGTON
Type: PERSON, Value: Obama
Type: ORG, Value: Trump
Type: MONEY, Value: billions of dollars
Type: NORP, Value: Americans
Type: ORG, Value: House
Type: NORP, Value: Republicans
Type: CARDINAL, Value: millions
Type: NORP, Value: Republicans
Type: NORP, Value: Republicans
Type: NORP, Value: Republicans
Type: GPE, Value: Obama
Type: DATE, Value: years
Type: PERSON, Value: Donald J. Trump’s
Type: NORP, Value: Republican
Type: ORG, Value: House
Type: NORP, Value: Republicans
Type: ORG, Value: Capitol Hill
Type: ORG, Value: Trump
Type: DATE, Value: at least late February
Type: ORG, Value: the United States Court of Appeals
Type: GPE, Value: the District of Columbia Circuit
Type: PERSON, Value: Obama
Type: ORG, Value: Congress
Type: PERSON, Value: Phillip J. Blando
Type: ORG, Value: Trump
Type: ORG, Value: Trump
Type: LAW, Value: the Affordable Care Act
Type: DATE, Value: 2015
Type: PERSON, Value: Rosemary M. Collyer
Type: ORG, Value: House
Type: NORP, Value: Republ

In [28]:
# we can visualize the entity of the entire article.

import spacy 
from spacy import displacy 

nlp = spacy.load("en") 
doc = nlp(artical) 
displacy.serve(doc, style="ent")



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [27]:
document1 = spacy_nlp(artical)

for ent in document1.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_, spacy.explain(ent.label_))

# 'In the above , ent is a Span object with various attributes:

# text gives the Unicode text representation of the entity.
# start_char denotes the character offset for the start of the entity.
# end_char denotes the character offset for the end of the entity.
# label_ gives the label of the entity.



WASHINGTON 0 10 GPE Countries, cities, states
Obama 110 115 PERSON People, including fictional
Trump 161 166 ORG Companies, agencies, institutions, etc.
billions of dollars 311 330 MONEY Monetary values, including unit
Americans 369 378 NORP Nationalities or religious or political groups
House 388 393 ORG Companies, agencies, institutions, etc.
Republicans 394 405 NORP Nationalities or religious or political groups
millions 543 551 CARDINAL Numerals that do not fall under another type
Republicans 604 615 NORP Nationalities or religious or political groups
Republicans 732 743 NORP Nationalities or religious or political groups
Republicans 808 819 NORP Nationalities or religious or political groups
Obama 920 925 GPE Countries, cities, states
years 1018 1023 DATE Absolute or relative dates or periods
Donald J. Trump’s 1043 1060 PERSON People, including fictional
Republican 1159 1169 NORP Nationalities or religious or political groups
House 1184 1189 ORG Companies, agencies, institutions, 

**Conclusion**

My approach is to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.